<a href="https://colab.research.google.com/github/ArshdeepKaur/Generative_AI/blob/main/nlp_information_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import os
import pandas as pd
import re
from spacy.matcher import Matcher

In [2]:
twitter_data = pd.read_csv("/content/Demonatization_tweets.csv", encoding='unicode_escape')

In [3]:
nlp=spacy.load('en_core_web_sm')

In [4]:
def print_data_summary(df):
    print(df.info())
    print(df.shape)
    print(df.head())
    print(df.tail())

In [5]:
print_data_summary(twitter_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14940 entries, 0 to 14939
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  14940 non-null  int64 
 1   text        14940 non-null  object
dtypes: int64(1), object(1)
memory usage: 233.6+ KB
None
(14940, 2)
   Unnamed: 0                                               text
0           1  RT @rssurjewala: Critical question: Was PayTM ...
1           2  RT @Hemant_80: Did you vote on #Demonetization...
2           3  RT @roshankar: Former FinSec, RBI Dy Governor,...
3           4  RT @ANI_news: Gurugram (Haryana): Post office ...
4           5  RT @satishacharya: Reddy Wedding! @mail_today ...
       Unnamed: 0                                               text
14935       14936  RT @saxenavishakha: Ghost of demonetization re...
14936       14937  N d modi fans-d true nationalists of the count...
14937       14938  RT @bharat_builder: Lol. Demonetization has fi..

In [6]:
twitter_data = twitter_data.rename(columns={"Unnamed: 0": "tweet_number", "text": "tweet"}, errors="raise")
print_data_summary(twitter_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14940 entries, 0 to 14939
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweet_number  14940 non-null  int64 
 1   tweet         14940 non-null  object
dtypes: int64(1), object(1)
memory usage: 233.6+ KB
None
(14940, 2)
   tweet_number                                              tweet
0             1  RT @rssurjewala: Critical question: Was PayTM ...
1             2  RT @Hemant_80: Did you vote on #Demonetization...
2             3  RT @roshankar: Former FinSec, RBI Dy Governor,...
3             4  RT @ANI_news: Gurugram (Haryana): Post office ...
4             5  RT @satishacharya: Reddy Wedding! @mail_today ...
       tweet_number                                              tweet
14935         14936  RT @saxenavishakha: Ghost of demonetization re...
14936         14937  N d modi fans-d true nationalists of the count...
14937         14938  RT @bharat_builder: 

In [7]:
#Look at the top 50 mentions in the dataset (mentions in this dataset refers to the @s)

def mention(x):
    found = re.findall(r'@\w+',x)
    if found:
        return found
    return None

mentions = twitter_data['tweet'].apply(lambda x: mention(x))
#print(mentions.head())

#combine all mentions in a list
mentions_list = []

for x in mentions:
    if x != None:
        mentions_list.extend(x)

#print(mentions_list)
mentions_count = pd.Series(mentions_list).value_counts().head(50)
print (mentions_count)

@evanspiegel        1311
@URautelaForever    1273
@narendramodi       1138
@gauravcsawant       541
@ModiBharosa         540
@DrKumarVishwas      358
@5Forty3             286
@rssurjewala         284
@rahulroushan        284
@PMOIndia            246
@centerofright       237
@ShashiTharoor       231
@DasShaktikanta      195
@ArvindKejriwal      185
@steve_hanke         179
@ashu3page           171
@YouTube             159
@kanimozhi           153
@RoflGandhi_         151
@AskAnshul           149
@Timcast             147
@Atheist_Krishna     135
@ippatel             116
@Joydas              114
@jamewils            114
@arunjaitley         114
@PIB_India           113
@FinMinIndia         104
@attomeybharti       103
@Joydeep_911         102
@PiyushGoyalOffc      95
@Memeghnad            95
@DrGPradhan           90
@dna                  85
@priyaakulkarni2      80
@arvindsubraman       80
@minimathur           79
@MIB_India            79
@Ra_THORe             75
@pGurus1              74


In [8]:
#Find the top 50 most frequently used hashtags (#)

def hashtag(x):
    found = re.findall(r'#\w+',x)
    if found:
        return found
    return None

hashtags = twitter_data['tweet'].apply(lambda x: hashtag(x))
#print(hashtags.head())

#combine all hashtags in a list
hashtags_list = []

for x in hashtags:
    if x != None:
        hashtags_list.extend(x)

#print(hashtags_list)
hashtags_count = pd.Series(hashtags_list).value_counts().head(50)
print (hashtags_count)

#Demonetization           3884
#demonetization           3783
#DeMonetization            920
#India                     356
#nitishkumar               257
#demo                      174
#GLvMI                     145
#CorruptionFreeIndia       103
#BlackMoney                 96
#Modi                       92
#VijayMallya                86
#IndiaFightsCorruption      79
#DEMONETIZATION             60
#BJP                        56
#NoMoneyYaar                53
#Demonetizat                47
#SonuNigam                  47
#DeMonetisation             45
#RatanTata                  44
#Insights                   44
#boycottsnapchat            44
#bulletin                   44
#RBI                        42
#NarendraModi               41
#LifeInsurance              34
#demonetisation             31
#YouTubeIsDead              30
#NMApp                      29
#MeraDeshBadalRaha          28
#PMModi                     26
#news                       25
#TrustBJP                   25
#Bypolls

In [9]:
# Find the sentences having mentions of Prime Minister
def find_names(text):

    names = []

    # spacy doc
    doc = nlp(text)

    # pattern
    pattern = [{'LOWER':'prime'},
              {'LOWER':'minister'},
              {'POS':'ADP','OP':'?'},
              {'POS':'PROPN'}]

    # Matcher class object
    matcher = Matcher(nlp.vocab)
    matcher.add("pm_names", [pattern])

    matches = matcher(doc)

    # finding patterns in the text
    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        #print(token)
        # append token to list
        names.append(str(token))

    return names

# apply function
#pm_names = twitter_data['tweet'].apply(find_names)

#extracting sentences with PM names
extracted_pm_names = []
for i in range(twitter_data.shape[0]):
    extracted_pm_names.append(find_names(twitter_data['tweet'][i]))

twitter_data['pm_name'] = extracted_pm_names

#extracting only the not null values
mention = []
for i in range(twitter_data.shape[0]):
    if twitter_data['pm_name'][i] != []:
        mention.append(twitter_data['pm_name'][i])

print(mention)

#extracting the sentences which have Prime Minister mentioned
sentences_pm = []
for i in range(twitter_data.shape[0]):
    if twitter_data['tweet'][i] != []:
        sentences_pm.append(twitter_data['tweet'][i])

print(sentences_pm)


[['prime minister sir'], ['Prime Minister #'], ['Prime minister Sri'], ['Prime minister Sri'], ['Prime Minister of India'], ['Prime Minister @narendramodi']]
["RT @rssurjewala: Critical question: Was PayTM informed about #Demonetization edict by PM? It's clearly fishy and requires full disclosure &amp;\x85", 'RT @Hemant_80: Did you vote on #Demonetization on Modi survey app?', 'RT @roshankar: Former FinSec, RBI Dy Governor, CBDT Chair + Harvard Professor lambaste #Demonetization.\r\n\r\nIf not for Aam Aadmi, listen to th\x85', 'RT @ANI_news: Gurugram (Haryana): Post office employees provide cash exchange to patients in hospitals #demonetization https://t.co/uGMxUP9\x85', 'RT @satishacharya: Reddy Wedding! @mail_today cartoon #demonetization #ReddyWedding https://t.co/u7gLNrq31F', '@DerekScissors1: India\x92s #demonetization: #Blackmoney a symptom, not the disease https://t.co/HSl6Ihj0Qe via @ambazaarmag', "RT @gauravcsawant: Rs 40 lakh looted from a bank in Kishtwar in J&amp;K. Third s

In [10]:
#Use the prepositions to extract relevant information from the tweets
def find_prepositions(text):

    sent = []

    # spacy doc
    doc = nlp(text)

    for token in doc:
        if token.pos_ == 'ADP':
            phrase = ''
            #if the head word is a noun
            if token.head.pos_ == 'NOUN':
                phrase += token.head.text
                phrase += ' '+token.text

            #check nodes to the right of the preposition
            for right_tok in token.rights:
                #append if it is a noun or a pronoun
                if right_tok.pos_ in ('NOUN', 'PROPN'):
                    phrase += ' '+right_tok.text
            count_spaces = 0
            for i in range(0, len(phrase)):
                if phrase[i] == " ":
                    count_spaces += 1
            if count_spaces > 2:
                #print("phrase after NOUN/PROPN", phrase, len(phrase))
                sent.append(phrase)
                print(sent)
    return sent

In [11]:
row_list = []
for i in range(twitter_data.shape[0]):
    tweet_number = twitter_data.loc[i]['tweet_number']
    tweet = twitter_data.loc[i]['tweet']
    preposition_data = find_prepositions(tweet)
    dict1 = {'tweet_number': tweet_number, 'tweet':tweet, 'preposition_data':preposition_data}
    row_list.append(dict1)
df_preposition = pd.DataFrame(row_list)
print(df_preposition.head())

['policy with o https://t.co/TPYJgXFBY8']
['Impact On sector https://t.co/aVcYTIhdtH']
['impact on # women']
['impact on # women']
['lots of bk U+20B9']
['MOVE BY PM @NARENDRAMODI']
['MOVE BY PM @NARENDRAMODI']
['demonstration against demonetization https://t.co/eQBuCBq6V1']
['demonetization By VIDEO U+279C']
['hours after demonetization https://t.co/IdMgdpx0sz']
['24k per week account']
['24k per week account']
['Impact On sector https://t.co/aVcYTIhdtH']
['Impact On sector https://t.co/aVcYTIhdtH']
['move by NarendraModi @rsprasad']
['support for # PM']
['effects of # Demonetization']
['support for # PM']
['support for # PM']
['support for # PM']
['support for # PM']
['support for # PM']
['support for # PM']
['support for # PM']
['support for # PM']
['decision for # Demonetization']
['support for # PM']
['support for # PM']
['Man from # Kurseong']
['failure of indians @ArvindKejriwal']
['failure of indians @ArvindKejriwal']
['poll on # https://t.co/2qGBFx3EJw']
['effects of # Demonet

In [ ]:
#print the non-empty matches
df_show = pd.DataFrame(columns = df_preposition.columns)
for row in range(len(df_preposition)):
    if df_preposition.loc[row]['preposition_data'] != 'None':
        df_show = df_show.append(df_preposition.loc[row,:])

df_show.reset_index(drop=True, inplace=True)

In [13]:
df_show.head(10)

,tweet_number,tweet,preposition_data
0,1,RT @rssurjewala: Critical question: Was PayTM ...,[]
1,2,RT @Hemant_80: Did you vote on #Demonetization...,[]
2,3,"RT @roshankar: Former FinSec, RBI Dy Governor,...",[]
3,4,RT @ANI_news: Gurugram (Haryana): Post office ...,[]
4,5,RT @satishacharya: Reddy Wedding! @mail_today ...,[]
5,6,@DerekScissors1: Indias #demonetization: #Bla...,[]
6,7,RT @gauravcsawant: Rs 40 lakh looted from a ba...,[]
7,8,RT @Joydeep_911: Calling all Nationalists to j...,[]
8,9,RT @sumitbhati2002: Many opposition leaders ar...,[]
9,10,National reform now destroyed even the essence...,[]
